In [ ]:
import pandas as pd

df=pd.read_csv('../project_files/movies.csv')

In [ ]:
df.shape

In [ ]:
# Drop duplicate rows
df.drop_duplicates(inplace=True)
df.shape

---

### Movie Table
`movie(id, original_title)`

In [ ]:
# id is primary key, drop rows with duplicate id
df.drop_duplicates(subset=['id'], inplace=True)

movies_table = df[['id','original_title']]
movies_table

In [ ]:
"""
INSERT INTO movie (id, original_title)
VALUES (135397, "Jurassic World");
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/movies_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/movies_table.sql", "a")  # append mode

for ind in movies_table.index:
    # original_title have single qoutes
    original_title = str(movies_table['original_title'][ind])
    original_title = original_title.replace("'", "''")
    file1.write(f"INSERT INTO movie (id, original_title) VALUES ({movies_table['id'][ind]},\'{original_title}\');\n")
file1.close()

### IMDb Details
`imdb_details(id, imdb_id, popularity, vote_count, vote_average)`

In [ ]:
imdb_df = df.copy(deep=True)

# id is primary key, drop rows with duplicate id
imdb_df.drop_duplicates(['imdb_id'], inplace=True)

imdb_table = imdb_df[['id', 'imdb_id', 'popularity', 'vote_count', 'vote_average']]

imdb_table

In [ ]:
df.imdb_id.unique().shape

In [ ]:
"""
INSERT INTO imdb_details (id, imdb_id, popularity, vote_count, vote_average)
VALUES (id, imdb_id, popularity, vote_count, vote_average);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/imdb_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/imdb_table.sql", "a")  # append mode

for ind in imdb_table.index:
    file1.write(f"INSERT INTO imdb_details (id, imdb_id, popularity, vote_count, vote_average) VALUES ({imdb_table['id'][ind]}, '{imdb_table['imdb_id'][ind]}', {imdb_table['popularity'][ind]}, {imdb_table['vote_count'][ind]}, {imdb_table['vote_average'][ind]});\n")
file1.close()

### Genre Details
`movie_genre(id, genre)`

In [ ]:
genre_df = df.copy(deep=True)
genre = genre_df[['id', 'genres']]
genre.loc[:, 'genres'] = genre.genres.apply(lambda name: str(name).split('|'))
genre.head(20)

In [ ]:
"""
INSERT INTO movie_genre (id, genre)
VALUES (id, genre);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/genre_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/genre_table.sql", "a")  # append mode

for ind in genre.index:
    for each_genre in genre['genres'][ind]:
        if each_genre=='nan':
            continue # escape nan values
        file1.write(f"INSERT INTO movie_genre (id, genre) VALUES ({genre['id'][ind]}, '{each_genre}');\n")
file1.close()

### Release Details
`release_details(id, release_date, release_year)`

In [ ]:
release_df = df.copy(deep=True)
release_details = release_df[['id', 'release_date', 'release_year']]

for ind in release_details.index:
    dd_mm_yy = release_details['release_date'][ind]
    year =  release_details['release_year'][ind]
    # f-string to turn one digit integer to two-digit > f"{a:02}"
    dd = f"{int(release_details['release_date'][ind].split('/')[1]):02}"
    mm = f"{int(release_details['release_date'][ind].split('/')[0]):02}"
    yyyy_mm_dd = f"{year}-{mm}-{dd}"
    # release_details['release_date'][ind] = yyyy_mm_dd
    release_details.loc[ind, "release_date"] = yyyy_mm_dd

#release_details.loc[:, 'release_date'] = release_details.release_date.apply(lambda x: x.replace('/', '-'))
release_details

In [ ]:
"""
INSERT INTO release_details (id, release_date, release_year)
VALUES (id, release_date, release_year);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/release_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/release_table.sql", "a")  # append mode

for ind in release_details.index:
    file1.write(f"INSERT INTO release_details (id, release_date, release_year) VALUES ({release_details['id'][ind]}, \'{release_details['release_date'][ind]}\', {release_details['release_year'][ind]});\n")
file1.close()

### Metadata
`metadata(id, keywords, tagline, runtime, homepage, overview)`

In [ ]:
metadata_df = df.copy(deep=True)
metadata = metadata_df[['id', 'keywords', 'tagline', 'runtime', 'homepage', 'overview']]
metadata.loc[:, 'keywords'] = metadata.keywords.apply(lambda name: str(name).replace('|', ', '))
metadata.loc[:, 'keywords'] = metadata.keywords.apply(lambda name: "{"+name+"}") #since keywords is charvar list

# Dealing with single qoutes in text fields
metadata.loc[:, 'keywords'] = metadata.keywords.apply(lambda text: str(text).replace("'", "''"))
metadata.loc[:, 'tagline'] = metadata.tagline.apply(lambda text: str(text).replace("'", "''"))
metadata.loc[:, 'homepage'] = metadata.homepage.apply(lambda text: str(text).replace("'", "''"))
metadata.loc[:, 'overview'] = metadata.overview.apply(lambda text: str(text).replace("'", "''"))


metadata.head()

In [ ]:
"""
INSERT INTO metadata (id, keywords, tagline, runtime, homepage, overview)
VALUES (id, keywords, tagline, runtime, homepage, overview);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/metadata_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/metadata_table.sql", "a")  # append mode

for ind in metadata.index:
    file1.write(f"INSERT INTO metadata (id, keywords, tagline, runtime, homepage, overview) VALUES ({metadata['id'][ind]}, \'{metadata['keywords'][ind]}\', \'{metadata['tagline'][ind]}\', {metadata['runtime'][ind]}, \'{metadata['homepage'][ind]}\', \'{metadata['overview'][ind]}\');\n")
file1.close()

In [ ]:
metadata.shape

### Finance Details
`finances(id, budget, revenue, budget_adj, revenue_adj)`

In [ ]:
# To deal with profit margin (revenue to budget ratio)
# we replace budget 0 with 1, to avoid 'division-by-zero' error and still represent proper profit margin
df.loc[:, 'budget'] = df.budget.apply(lambda x: x if x!=0 else 1)

# And now the finances table
finances = df[['id','budget', 'revenue', 'budget_adj', 'revenue_adj']]
finances

In [ ]:
"""
INSERT INTO finances (id, budget, revenue, budget_adj, revenue_adj)
VALUES (id, budget, revenue, budget_adj, revenue_adj);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/finances_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/finances_table.sql", "a")  # append mode

for ind in finances.index:
    file1.write(f"INSERT INTO finances (id, budget, revenue, budget_adj, revenue_adj) VALUES ({finances['id'][ind]}, {finances['budget'][ind]}, {finances['revenue'][ind]}, {finances['budget_adj'][ind]}, {finances['revenue_adj'][ind]});\n")
file1.close()

### Movie-Director
`directed(id, director_name)`

In [ ]:
directed_df = df.copy(deep=True)
directed = directed_df[['id','director']]
print(directed.head(20))

In [ ]:
directed.loc[:, 'director'] = directed.director.apply(lambda name: str(name).split('|'))
directed.head(20)

In [ ]:
"""
INSERT INTO directed (id, director_name)
VALUES (id, director_name);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/director_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/director_table.sql", "a")  # append mode

for ind in directed.index:
    for each_director in directed['director'][ind]:
        # There are single qoutes in director names, so we replace them with two consecutive single qoutes to escape
        each_director = each_director.replace("'", "''")
        if each_director=='nan':
            continue # removes nan values
        file1.write(f"INSERT INTO directed (id, director_name) VALUES ({directed['id'][ind]}, '{each_director}');\n")
file1.close()

### Casting Details
`movie_cast(id, actor_name)`

In [ ]:
casting_df = df.copy(deep=True)
movie_cast = casting_df[['id','cast']]
print(movie_cast.head(20))

In [ ]:
movie_cast.loc[:, 'cast'] = movie_cast.cast.apply(lambda name: str(name).split('|'))
movie_cast.head()

In [ ]:
# There are duplicates in actors lists
def remove_dups(x):
  return list(dict.fromkeys(x))

movie_cast.loc[:, 'cast'] = movie_cast.cast.apply(lambda actor_list: remove_dups(actor_list))

In [ ]:
"""
INSERT INTO cast (id, actor_name)
VALUES (id, actor_name);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/actors_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/actors_table.sql", "a")  # append mode

for ind in movie_cast.index:
    for each_actor in movie_cast['cast'][ind]:
        if each_actor=='nan':
            continue # removes nan values
        # There are single qoutes in actor names, so we replace them with two consecutive single qoutes to escape
        each_actor = each_actor.replace("'", "''")
        file1.write(f"INSERT INTO movie_cast (id, actor_name) VALUES ({movie_cast['id'][ind]}, '{each_actor}');\n")
file1.close()

### Production Details
`production(id, production_company)`

In [ ]:
production_df = df.copy(deep=True)
production = production_df[['id','production_companies']]
print(production.head(20))

In [ ]:
production.loc[:, 'production_companies'] = production.production_companies.apply(lambda name: str(name).split('|'))
production.head(20)

In [ ]:
# There are duplicates in production companies lists
production.loc[:, 'production_companies'] = production.production_companies.apply(lambda companies_list: remove_dups(companies_list))

In [ ]:
"""
INSERT INTO production (id, production_company)
VALUES (id, production_company);
"""
# First empty the file if it already exist, else create an empty file
file1 = open("../setup/producers_table.sql", "w")
file1.write("")
file1.close()

file1 = open("../setup/producers_table.sql", "a")  # append mode

for ind in production.index:
    for each_company in production['production_companies'][ind]:
        # There are single qoutes in company names, so we replace them with two consecutive single qoutes to escape
        each_company = each_company.replace("'", "''")
        if each_company=='nan':
            continue # remove nan values
        file1.write(f"INSERT INTO production (id, production_company) VALUES ({production['id'][ind]}, '{each_company}');\n")
file1.close()